# Download

Berikut adalah contoh script untuk mengunduh data GDELT dengan country code ID. Selanjutnya dari csv bisa dipindah ke mysql

In [ ]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'

# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits
file_list = [x for x in link_list if str.isdigit(x[0:4])]
print(file_list)

infilecounter = 0
outfilecounter = 0

import os.path
import urllib
import zipfile
import glob
import operator

local_path = 'E:/GDELT/'

fips_country_code = 'ID'

for compressed_file in file_list[infilecounter:]:
    print(compressed_file)

    # if we dont have the compressed file stored locally, go get it. Keep trying if necessary.
    while not os.path.isfile(local_path + compressed_file):
        print('downloading, '),
        urllib.urlretrieve(url=gdelt_base_url + compressed_file,
                           filename=local_path + compressed_file)

    # extract the contents of the compressed file to a temporary directory
    print('extracting,'),
    z = zipfile.ZipFile(file=local_path + compressed_file, mode='r')
    z.extractall(path=local_path + 'tmp/')

    # parse each of the csv files in the working directory,
    print('parsing,'),
    for infile_name in glob.glob(local_path + 'tmp/*'):
        outfile_name = local_path + 'country/' + fips_country_code + '%04i.tsv' % outfilecounter

        # open the infile and outfile
        with open(infile_name, mode='r') as infile, open(outfile_name, mode='w') as outfile:
            for line in infile:
                # extract lines with our interest country code
                if fips_country_code in operator.itemgetter(51, 37, 44)(line.split('\t')):
                    outfile.write(line)
            outfilecounter += 1

        # delete the temporary file
        os.remove(infile_name)
    infilecounter += 1
    print('done')

# Scraping GDELT
Berikut adalah kode contoh untuk melakukan scraping dengan library newspaper. Skrip ini akan berjalan jika data dari GDELT telah tersimpan ke MySQL

In [ ]:
from newspaper import Article
import pymysql.cursors
import sys

offset = sys.argv[1]
limit = sys.argv[2]
# year = sys.argv[3]

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='',
                             db='news',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

hdr = {'User-Agent': 'Mozilla/5.0'}

with connection.cursor() as cursor:
    sql = "SELECT * FROM `gdelt_content` WHERE (TITLE IS NULL OR TRIM(TITLE) = '') LIMIT "+str(offset)+","+str(limit)
    cursor.execute(sql)
    print(sql)
    result = cursor.fetchall()
    for r in result:
        try:
            r["URL"] = r["URL"].split('<UDIV>')
            print(r["URL"][0])
            article = Article(r["URL"][0])
            article.download()
            article.parse()
            with connection.cursor() as cursor:
                sql = "UPDATE `gdelt_content` SET title=%s,content=%s WHERE GLOBALEVENTID=%s"
                cursor.execute(sql, (article.title.encode('utf-8'), article.text.encode('utf-8'),r["GLOBALEVENTID"]))
                connection.commit()
        except:
            pass

Berikut adalah contoh untuk melakukan pickling data GDELT

In [ ]:
import glob
import pandas as pd

local_path = 'E:/GDELT/'
fips_country_code = 'ID'
# Get the GDELT field names from a helper file
colnames = pd.read_excel('CSV.header.fieldids.xlsx', sheetname='Sheet1',
                         index_col='Column ID', parse_cols=1)['Field Name']

# Build DataFrames from each of the intermediary files
files = glob.glob(local_path + 'country/' + fips_country_code + '*')
DFlist = []
for active_file in files:
    print active_file
    DFlist.append(pd.read_csv(active_file, sep='\t', header=None, dtype=str,
                              names=colnames, index_col=['GLOBALEVENTID']))
    break

# Merge the file-based dataframes and save a pickle
DF = pd.concat(DFlist)
DF.to_pickle(local_path + 'backupsmall' + fips_country_code + '.pickle')

# once everythin is safely stored away, remove the temporary files
# for active_file in files:
#     os.remove(active_file)